In [53]:
from tvDatafeed import TvDatafeed, Interval


In [54]:
username = 'amiramirmashgh'
password = '1234Asdf'

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [55]:
# get data
df_XUD = tv.get_hist(symbol='EURUSD',exchange='FXOPEN',interval=Interval.in_5_minute,n_bars=50000)
#correct timing
import pandas as pd
# offset = pd.Timedelta(hours=1)
# df_XUD.index = df_XUD.index - offset
print(df_XUD)

                            symbol     open     high      low    close  \
datetime                                                                 
2023-08-28 00:30:00  FXOPEN:EURUSD  1.07958  1.07961  1.07958  1.07959   
2023-08-28 00:35:00  FXOPEN:EURUSD  1.07959  1.07959  1.07939  1.07952   
2023-08-28 00:40:00  FXOPEN:EURUSD  1.07952  1.07961  1.07949  1.07954   
2023-08-28 00:45:00  FXOPEN:EURUSD  1.07954  1.07963  1.07945  1.07953   
2023-08-28 00:50:00  FXOPEN:EURUSD  1.07959  1.07961  1.07954  1.07954   
...                            ...      ...      ...      ...      ...   
2023-09-23 00:05:00  FXOPEN:EURUSD  1.06429  1.06435  1.06428  1.06433   
2023-09-23 00:10:00  FXOPEN:EURUSD  1.06434  1.06453  1.06434  1.06449   
2023-09-23 00:15:00  FXOPEN:EURUSD  1.06450  1.06453  1.06442  1.06451   
2023-09-23 00:20:00  FXOPEN:EURUSD  1.06452  1.06454  1.06439  1.06440   
2023-09-23 00:25:00  FXOPEN:EURUSD  1.06440  1.06474  1.06431  1.06432   

                       volume  
datet

In [56]:
# Add SMA 20
window_size = 20
df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

# ADD diff and color off the candle

# Calculate the difference between current and previous day's closing price
df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

# Add a new column to indicate green (1) or red (0)
df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')



# Assuming your DataFrame is named df
short_window = 12  # Short-term EMA
long_window = 26   # Long-term EMA
signal_window = 9  # Signal line EMA

# Calculate Short-term EMA
df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

# Calculate Long-term EMA
df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

# Calculate MACD line
df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

# Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

# Calculate MACD Histogram
df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

# Determine MACD trend direction (Above/Below Zero Line)
df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

# Determine Signal_Line trend direction (Above/Below Zero Line)
df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


# Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


# Function to determine color based on conditions
def determine_color(row):
    if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
        return 'green'  # Signal Line is below MACD and Histogram is increasing
    elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
        return 'light green'  # Signal Line is above MACD and Histogram is decreasing
    elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
        return 'red'  # Signal Line is above MACD and Histogram is decreasing
    else:
        return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

# Calculate the shift of the MACD_Histogram column
df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

# Apply the determine_color function to create the color column
df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

# Function to determine color based on conditions
def determine_color_finisher(row):
    if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
        return 'red'  # fix first hist candle
    elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
        return 'green'  # fix first hist candle
    else:
        return  row['Hist_color_V2'] # return defulat

# Calculate the shift of the MACD_Histogram column
df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

# Advaned level: GOD bless me
df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


# Create a cross counter column
counter = 0
counter_list = []
prev_color = None

for color in df_XUD['Hist_color_V1']:
    if prev_color is None or color == prev_color:
        counter += 1
    else:
        counter = 1
    counter_list.append(counter)
    prev_color = color

df_XUD['cross_counter'] = counter_list

def determine_candle_from_sma20(row):
    if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
        return 'buy' 
    elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
        return 'sell'  
    else:
        return  '' # return defulat

df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

def determine_distance_sma20(row):
    if (row['inside candle (rule 1)'] == 'buy'):
        return (row['high'] - row['SMA 20'] ) 
    elif (row['inside candle (rule 1)'] == 'sell') :
        return row['SMA 20'] - row['low']
    else:
        return  'no position' # return defulat

df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


# from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles 
df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

def determine_rule2(row):
    if row['distance SMA 20 from high or close'] == 'no position':
        return "False"
    elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
        return "True"  
    else:
        return  'False'

df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)  

def determine_rule3(row):
    if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
        return "buy"
    elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
        return "Half_buy"  
    elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
        return "sell"  
    else:
        return  'Half_sell'

df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1) 

#for long sabz kamrang and for short germez kamrang?
def determine_rule4_5(row):
    if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
        return "buy"
    elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green'  ) :
        return "buy"
    elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
        return "buy"
    elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
        return "sell"
    elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red') :
        return "sell"
    elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
        return "sell"

    else:
        return  'unknown'

df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1) 



df_XUD['SL_sell_shifted1'] = df_XUD['high'].shift(1)
df_XUD['SL_sell_shifted2'] = df_XUD['high'].shift(2)
df_XUD['SL_sell_shifted3'] = df_XUD['high'].shift(3)
df_XUD['SL_sell_shifted4'] = df_XUD['high'].shift(4)

df_XUD['SL_buy_shifted1'] = df_XUD['low'].shift(1)
df_XUD['SL_buy_shifted2'] = df_XUD['low'].shift(2)
df_XUD['SL_buy_shifted3'] = df_XUD['low'].shift(3)
df_XUD['SL_buy_shifted4'] = df_XUD['low'].shift(4)

def determine_SL_buy(row):
   return min(row['SL_buy_shifted1'],row['SL_buy_shifted2'],row['SL_buy_shifted3'],row['SL_buy_shifted4']) - 0.0001

def determine_SL_sell(row):
   return max(row['SL_sell_shifted1'],row['SL_sell_shifted2'],row['SL_sell_shifted3'],row['SL_sell_shifted4']) + 0.0001

df_XUD['SL_sell'] = df_XUD.apply(determine_SL_sell, axis=1) 
df_XUD['SL_buy'] = df_XUD.apply(determine_SL_buy, axis=1) 


def determine_entry_point_buy(row):
   return row["high"] + 0.0001

def determine_entry_point_sell(row):
   return row["low"] - 0.0001




df_XUD['entry_sell'] =  df_XUD.apply(determine_entry_point_sell, axis=1) 
df_XUD['entry_buy'] =  df_XUD.apply(determine_entry_point_buy, axis=1) 


def determine_TP_buy(row):
   return 1.5 * (row["entry_buy"] - row["SL_buy"]) + row["entry_buy"]

def determine_TP_sell(row):
   return  row["entry_sell"]  -  1.5 * (row["SL_sell"] -   row["entry_sell"])

df_XUD['TP_sell'] = df_XUD.apply(determine_TP_sell, axis=1) 
df_XUD['TP_buy'] = df_XUD.apply(determine_TP_buy, axis=1) 


def determine_finall_desions(row):
    if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= 0.0004 ):
        return "buy"
    elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"  and (row["SL_sell"] -   row["entry_sell"]) >= 0.0004 ):
        return "sell"
    elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell" and (row["SL_sell"] -   row["entry_sell"]) >= 0.0004 ):
        return "sell_half"
    elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy" and (row["entry_buy"] - row["SL_buy"]) >= 0.0004  ):
        return "buy_half"
    else :
        return ""

df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1) 

# check don't buy or sell it twice for the next 3 candles
# counter_buy =0
# counter_sell =0
# for i in range(len(df_XUD)):

#     if df_XUD['finall_desions'][i] == "buy" or df_XUD['finall_desions'][i] == "buy_half":
#         counter_buy = 3
#     if df_XUD['finall_desions'][i] == "sell" or df_XUD['finall_desions'][i] == "sell_half":
#         counter_sell = 3


# check we don't get out positions because of our entry point don't touch 3 candles or we get opposite candle and break sma 20
counter_stoped =0
counter_in = 0
counter_win = 0
tp_sell = 0
tp_buy = 0
sl_sell = 0
sl_buy = 0
win_pip =0
lose_pip =0
def check_position_buy(index,tp_buy,sl_buy):
    for i in range(index, len(df_XUD)):
        if sl_buy > df_XUD['low'][i]:
            return 1,0,i,"sl"
        if tp_buy < df_XUD['high'][i]:
            return 0,1,i,"tp"
        return 0,0,i,"s"

def check_position_sell(index,tp_sell,sl_sell):
    for i in range(index, len(df_XUD)):
        if sl_sell < df_XUD['high'][i]:
            return 1,0,i,"sl"
        if tp_sell > df_XUD['low'][i]:
            return 0,1,i,"tp"
        return 0,0,i,"s"


index_closed_sl = []
index_closed_tp = []

counter_buy =0
counter_sell =0
counter_buy_to_del_duplicated =0
counter_sell_to_del_duplicated =0
entry_price_buy = 0
entry_price_sell = 0
df_XUD['touched'] = None 
for i in range(len(df_XUD)):
    if counter_buy > 0:
        if (df_XUD['inside candle (rule 1)'][i] == "sell"):
            counter_buy = 0
            counter_buy_to_del_duplicated = 0
            df_XUD['touched'][i] = "deleted_buy"
    if counter_sell > 0:
        if (df_XUD['inside candle (rule 1)'][i] == "buy"):
            counter_sell = 0
            counter_sell_to_del_duplicated = 0
            df_XUD['touched'][i] = "deleted_sell"


    if counter_buy > 0:
        if df_XUD['high'][i] > entry_price_buy:
            df_XUD['touched'][i] = "IN"
            temp_stoped,temp_win,indexx,kind = check_position_buy(i,tp_buy , sl_buy)
            counter_stoped = counter_stoped +temp_stoped
            counter_win = counter_win + temp_win
            if kind == 'sl' :
                index_closed_sl.append(indexx)
                lose_pip=abs(entry_price_buy - sl_buy) +lose_pip
            elif kind == 'tp' :
                index_closed_tp.append(indexx)
                win_pip=abs(entry_price_buy - tp_buy) +win_pip

            temp_win,temp_stoped = 0,0
            counter_in = counter_in +1
            counter_buy = 0
    if counter_sell > 0 :
        if df_XUD['low'][i] < entry_price_sell:
            df_XUD['touched'][i] = "IN"
            temp_stoped,temp_win,indexx,kind =check_position_sell(i,tp_sell , sl_sell)
            if kind == 'sl' :
                index_closed_sl.append(indexx)
                lose_pip=abs(entry_price_sell - sl_sell) +lose_pip
            elif kind == 'tp' :
                index_closed_tp.append(indexx)
                win_pip=abs(entry_price_sell - tp_sell) +win_pip
            counter_stoped = counter_stoped + temp_stoped
            counter_win = counter_win + temp_win
            temp_win,temp_stoped = 0,0
            counter_in = counter_in +1
            counter_sell = 0

    if counter_buy_to_del_duplicated > 0 and  (df_XUD['finall_desions'][i] == "buy" or df_XUD['finall_desions'][i] == "buy_half"):
        df_XUD['finall_desions'][i] = "nothing"
    counter_buy_to_del_duplicated = counter_buy_to_del_duplicated - 1
    counter_buy = counter_buy -1

    if counter_sell_to_del_duplicated > 0 and  (df_XUD['finall_desions'][i] == "sell" or df_XUD['finall_desions'][i] == "sell_half"):
        df_XUD['finall_desions'][i] = "nothing"
    counter_sell_to_del_duplicated = counter_sell_to_del_duplicated - 1
    counter_sell = counter_sell - 1


    if df_XUD['finall_desions'][i] == "buy" or df_XUD['finall_desions'][i] == "buy_half":
        counter_buy = 3
        counter_buy_to_del_duplicated = 3
        entry_price_buy = df_XUD['entry_buy'][i]
        sl_buy =  df_XUD['SL_buy'][i]
        tp_buy =  df_XUD['TP_buy'][i]

    if df_XUD['finall_desions'][i] == "sell" or df_XUD['finall_desions'][i] == "sell_half":
        counter_sell = 3
        counter_sell_to_del_duplicated = 3
        entry_price_sell = df_XUD['entry_sell'][i]
        sl_sell =  df_XUD['SL_sell'][i]
        tp_sell =  df_XUD['TP_sell'][i]

df_XUD['finall_closed'] = ''
for index in index_closed_sl:
    df_XUD['finall_closed'][index] = "hit sl"

for index in index_closed_tp:
        df_XUD['finall_closed'][index] = "hit tp"



# Deleting multiple columns
columns_to_delete = ["SL_buy_shifted4","SL_buy_shifted3","SL_buy_shifted2","SL_buy_shifted1","SL_sell_shifted4","SL_sell_shifted3","SL_sell_shifted2","SL_sell_shifted1","Hist_color_V1", "MACD_Histogram_shifted", "Hist_color_V2" , "Hist_color_V2_shifted","candle_size_shifted4","candle_size_shifted3","candle_size_shifted2","candle_size_shifted1"]
df_XUD = df_XUD.drop(columns_to_delete, axis=1)

print(df_XUD)


# Save the DataFrame to a CSV file
df_XUD.to_csv('output_eur.csv', index=True)

print("DataFrame saved to 'output.csv'")

C:\Users\M.M\AppData\Local\Temp\ipykernel_22564\3995204181.py:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_XUD['finall_desions'][i] = "nothing"
C:\Users\M.M\AppData\Local\Temp\ipykernel_22564\3995204181.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_XUD['touched'][i] = "IN"
C:\Users\M.M\AppData\Local\Temp\ipykernel_22564\3995204181.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_XUD['touched'][i] = "deleted_

                            symbol     open     high      low    close  \
datetime                                                                 
2023-08-28 00:30:00  FXOPEN:EURUSD  1.07958  1.07961  1.07958  1.07959   
2023-08-28 00:35:00  FXOPEN:EURUSD  1.07959  1.07959  1.07939  1.07952   
2023-08-28 00:40:00  FXOPEN:EURUSD  1.07952  1.07961  1.07949  1.07954   
2023-08-28 00:45:00  FXOPEN:EURUSD  1.07954  1.07963  1.07945  1.07953   
2023-08-28 00:50:00  FXOPEN:EURUSD  1.07959  1.07961  1.07954  1.07954   
...                            ...      ...      ...      ...      ...   
2023-09-23 00:05:00  FXOPEN:EURUSD  1.06429  1.06435  1.06428  1.06433   
2023-09-23 00:10:00  FXOPEN:EURUSD  1.06434  1.06453  1.06434  1.06449   
2023-09-23 00:15:00  FXOPEN:EURUSD  1.06450  1.06453  1.06442  1.06451   
2023-09-23 00:20:00  FXOPEN:EURUSD  1.06452  1.06454  1.06439  1.06440   
2023-09-23 00:25:00  FXOPEN:EURUSD  1.06440  1.06474  1.06431  1.06432   

                       volume    SMA 

In [57]:
print(index_closed_sl)

[]


In [58]:
print(df_XUD["finall_closed"][129])

In [59]:
print(counter_stoped,counter_win)

0 0


In [60]:
print(counter_in)

108


In [61]:
print(lose_pip, win_pip)

0 0.13453499999999696


In [62]:
print(lose_pip*100000, win_pip*100000)

0 13453.499999999696


In [63]:
df_XUD.dropna(subset=['finall_desions'], inplace=True)
df_XUD.to_csv('output1.csv', index=True)

In [64]:
df_cleaned = df_XUD[df_XUD['finall_desions'] != ""]
df_cleaned.to_csv('outpu2.csv', index=True)

In [65]:

df_cleaned = df_XUD[df_XUD['finall_desions'] != ""]
df_cleaned.to_csv('outpu3.csv', index=True)